## Bilateral filter

### Обработка изображения на CPU

In [1]:
#Считываем изображение и указываем значение сигма
import numpy as np
import cv2
image = cv2.imread('sample_data/woman.bmp', cv2.IMREAD_GRAYSCALE)
gg=200

In [2]:
#Метод для рассчёта двустороннего фильтра
def bilateralFilterCPU(image, gg):
    result = np.zeros(image.shape)
    for a in range(1, image.shape[0]-1):
        for b in range(1, image.shape[1]-1):
            SumK = 0
            SumHP = 0    
            for d in range(a-1, a+2):
                for c in range(b-1, b+2):

                    r = np.exp((-(image[d, c] - image[a,b])** 2) / gg ** 2) 
                    
                    g= np.exp(-((d - a) ** 2 + (c - b) ** 2) / gg ** 2)

                    SumK += g * r
                    SumHP += g * r * image[d, c]

            SumH = SumHP / SumK
            result[a, b] = SumH

    return m(result)

In [3]:
# метод для обработки граничных пикселей, так как основной метод их не захватывает
def m(image):
    for a in range(0, image.shape[0]):
        image[a,0]=image[a,1]
        image[a,image.shape[1]-1]=image[a,image.shape[1]-2]
        
    for b in range(0, image.shape[1]):
        image[0,b]=image[1,b]
        image[image.shape[0]-1,b]=image[image.shape[0]-2,b] 
    return image


In [4]:
#Запускаем обработку изображения на cpu 
import time
start1 = time.clock()

result_image=bilateralFilterCPU(image,gg)

end1 = time.clock() - start1

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:11: RuntimeWarning: overflow encountered in ubyte_scalars
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:7: DeprecationWarning: time.clock has been deprecated in Python 3.3 and will be removed from Python 3.8: use time.perf_counter or time.process_time instead
  import sys


In [5]:
#Записываем полученное изображение
cv2.imwrite('sample_data/res1_woman.bmp', result_image)

True

In [6]:
#Выводим время работы на cpu
print("Время обработки изображения на CPU -", end1)

Время обработки изображения на CPU - 22.033977


### Теперь всё тоже самое, но на GPU

In [7]:
#Для начала устанавливаем pycuda
!pip install pycuda

In [12]:
#Подгружаем всё необходимое
from pycuda import driver, compiler
import pycuda.autoinit
BLOCK_SIZE = 16

In [9]:
#Пишем функцию ядра
from pycuda.compiler import SourceModule
calculate_bilateral_GPU = SourceModule("""
texture<unsigned int, 2> tex;

__global__ void kernel(unsigned int * __restrict__ image, const int M, const int N, const float gg)
{
    //получаем номер нити
    const int x = threadIdx.x + blockDim.x * blockIdx.x;
    const int y = threadIdx.y + blockDim.y * blockIdx.y;

    //проверяем что не вышли за рамки изображения
    if ((x < M) && (y < N)) {

        //это центральный пиксель, который мы обрабатываем
        float fa0 = tex2D(tex, x, y);
        float SumK = 0;
        float SumHP = 0;

        for (int a = x-1; a <= x+1; a++){
            for (int b = y-1; b <= y+1; b++){

                //получаем пиксели, которые располагаются вокруг центрального
                float fai = tex2D(tex, b, a);

                //далее рассчитываем всё по формулам, как и в методе для cpu
                float r = exp(-pow((fai - fa0), 2) / pow(gg, 2));
                
                float g = exp(-(pow(b - x, 2) + pow(a - y, 2)) / pow(gg, 2));
               
                SumK += g*r;
                SumHP += g*r*tex2D(tex, b, a);
            }
        }
        image[x*N + y] = SumHP / SumK;
    }
}""")

kernel = calculate_bilateral_GPU.get_function("kernel")

In [10]:
#Задаём сетку
M,N=image.shape
grid = (int(np.ceil(M/BLOCK_SIZE)),int(np.ceil(N/BLOCK_SIZE)))

start2 = time.time()

#Копируем данные в текстуру
cu_tex = calculate_bilateral_GPU.get_texref("tex")
driver.matrix_to_texref(image.astype(np.int32), cu_tex, order="C")

#Запускаем функцию ядра
result_image2 = np.zeros((M,N), dtype=np.int32)
kernel(driver.Out(result_image2), 
       np.int32(M), np.int32(N), 
       np.float32(gg),
       texrefs=[cu_tex], 
       block=(BLOCK_SIZE,BLOCK_SIZE,1), 
       grid=grid)

end2 = time.time() - start2

#Сохраняем новое изображение
cv2.imwrite('sample_data/res2_woman.bmp', result_image2.astype(np.uint8))

True

In [11]:
print("Время обработки изображения на GPU -", end2)

Время обработки изображения на GPU - 0.006096601486206055
